In [ ]:
from muon_sim import mcmc
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def gauss(x):
	"""Just an unormalized gaussian density function"""
	return np.exp(-np.power(x, 2) / 0.5)

In [ ]:
x = np.random.uniform(-2, 2, 1024)
npdf = mcmc.pdf_normed(gauss, (-10, 10))
x = mcmc.walk(x, 1024, 0.1, npdf)  # Converge trash run

# Find best sigma 
ie the sigma for wich acceptance rate is ~ 60%

In [ ]:
sigma = np.linspace(0.1, 2, 64)
arate = np.zeros(64)
for i, s in enumerate(sigma):
    mean = 0
    for _ in range(100):
        x = mcmc.walk(x, 1, s, npdf)
        y = mcmc.walk(x, 1, s, npdf)
        mean += np.sum(np.isclose(x, y)) / x.shape[0]
    arate[i] = mean / 100


diff = np.abs(arate - 0.6)
min_index = np.argmin(diff)
BEST_SIGMA = sigma[min_index]

In [ ]:
plt.plot(sigma, arate)
BEST_SIGMA, arate[min_index]

In [ ]:
steps = np.arange(200, dtype="int")
correlation = np.zeros(200)
sigma = BEST_SIGMA
for n in steps:
    x = mcmc.walk(x, 1, sigma, npdf)
    y = mcmc.walk(x, n, sigma, npdf)
    correlation[n] = np.corrcoef(x, y)[0, 1]

Après un ncorr suffisant la valeur de la correlation entre deux walker oscille dans des valeurs proche de zéro
Graphiquement on estime qu'il est inutile de chercher à avoir une corr inférieur à ATOL

In [ ]:
ATOL = 0.05
for i, c in enumerate(correlation):
    if c < ATOL:
        break
NCOOR = i

In [ ]:
plt.plot(steps, correlation)
NCOOR, correlation[NCOOR]